In [1]:
import sys
sys.path.append(r'C:\Users\damod\OneDrive\Área de Trabalho\Projetos Python\dataprocessing')

Importanção das bibliotecas

In [2]:
from concurrent.futures import ThreadPoolExecutor
import dataprocessing.datacolect as dc
import re
import pandas as pd
import numpy as np

# 1. Sobre a coleta no ***Metacritic***

# 2. Rotina de ***scraping***

## 2.1. Coleta dos ***hrefs*** para acessar as ***urls*** dos jogos

É necessário coletar o *link* para cada jogo em cada uma das páginas do ***Metacritic*** referentes ao ***Xbox One***. Há, para essa plataforma, 319 páginas.

A ***url*** abaixo será usada em cada iteração, adicionando-se apenas o número correto da página.

In [3]:
url = ('https://www.metacritic.com/browse/game/xbox-one/all/all-time/new/'
       '?platform=xbox-one&releaseYearMin=1910&releaseYearMax=2023&page=')

In [4]:
def get_last_page(url: str):
    soup_class = dc.Soup(url)
    soup = soup_class.get_soup()
    scrape = dc.Scrape()

    last_page_soup = scrape.find_elements(
        tag='span', timeout=0, soup=soup,
        tag_class=('c-navigationPagination_itemButtonContent u-flexbox u-flexbox-alignCenter u-flexbox-justifyCenter')
    )
    last_page = int(re.sub(r'^\s+|\s+$', '', last_page_soup[-2].get_text(), flags=re.MULTILINE))

    return last_page

In [5]:
last_page = get_last_page(url)

In [6]:
def get_hrefs(url: str, last_page: int, init_page: int=1):
    hrefs = list() # lista em que o href de cada jogo será inserido.

    for page in range(init_page, last_page+1): # Para iterar até a última página devemos adicionar 1.
        # Instancia-se o objeto BeautifulSoup:
        soup_class = dc.Soup(url + str(page))
        soup = soup_class.get_soup()

        # Classe para realização do scraping:
        scrape = dc.Scrape()

        # Objeto BeautifulSoup com os cartões para os jogos na página:
        game_cards = scrape.find_elements(
            tag='div', tag_class='c-finderProductCard c-finderProductCard-game',
            timeout=0.05, soup=soup
        )
        # Itera-se cada cartão para obtenção do href:
        for game in game_cards:
            href = scrape.find_element(
                tag='a', tag_class='c-finderProductCard_container g-color-gray80 u-grid',
                timeout=0, soup=game
            )
            href = href['href']
            hrefs.append(href)
    return hrefs

In [7]:
with ThreadPoolExecutor(max_workers=3) as executor:
    games_hrefs = executor.submit(get_hrefs, url, last_page)
games_hrefs = games_hrefs.result()

## 2.2. Coleta das informações sobre os jogos

In [8]:
metacritic_url = 'https://www.metacritic.com'

In [15]:
def get_data_games(hrefs: list, metacritic_url: str):
    data_games = list()
    data_game = dict()
    
    for href in hrefs:
        soup_class = dc.Soup(metacritic_url + str(href))
        soup = soup_class.get_soup()
        scrape = dc.Scrape()

        name_soup = scrape.find_element(
            tag='div', timeout=0.1, soup=soup,
            tag_class='c-productHero_title g-inner-spacing-bottom-medium g-outer-spacing-top-medium'
        )
        name = re.sub(r'^\s+|\s+$', '', name_soup.get_text(), flags=re.MULTILINE)

        try:
            tbd_soup = scrape.find_element(
                tag='span', timeout=0, soup=soup,
                tag_class=('c-productHero_scoreRemainder u-inline-block g-color-gray80 '
                           'g-text-xsmall g-outer-spacing-bottom-large')
            )
            tbd = re.sub(r'^\s+|\s+$', '', tbd_soup.get_text(), flags=re.MULTILINE)
            condition = ['Critic reviews are not available', 'Available after 4 critic reviews']
            
            if tbd in condition:
                score = 'tbd'
            else:
                pass
        except (AttributeError, UnboundLocalError) as error:
            try:
                score_box_soup = scrape.find_element(
                            tag='div', timeout=0, soup=soup,
                            tag_class=("c-siteReviewScore u-flexbox-column u-flexbox-alignCenter"
                                    " u-flexbox-justifyCenter g-text-bold c-siteReviewScore_green"
                                    " g-color-gray90 c-siteReviewScore_medium")
                )
                score_soup = scrape.find_element(
                    tag='span', timeout=0, soup=score_box_soup           
                )
                score = int(score_soup.get_text())
            except AttributeError:
                score_box_soup = scrape.find_element(
                            tag='div', timeout=0, soup=soup,
                            tag_class=('c-siteReviewScore u-flexbox-column u-flexbox-alignCenter u-flexbox-justifyCenter g-text-bold c-siteReviewScore_red g-color-white c-siteReviewScore_medium')
                )
                score_soup = scrape.find_element(
                    tag='span', timeout=0, soup=score_box_soup           
                )
                score = int(score_soup.get_text())
        data_game['name'] = name
        data_game['metascore'] = score

        data_games.append(data_game.copy())
    return data_games

In [16]:
with ThreadPoolExecutor(max_workers=3) as executor:
    metacritic = get_data_games(games_hrefs, metacritic_url)

In [17]:
metacritic = pd.DataFrame(metacritic)

In [18]:
metacritic

,name,metascore
0,House Flipper: Farm,tbd
1,Transformers: Earthspark - Expedition,tbd
2,The Super Penguboy,tbd
3,Chemically Bonded,tbd
4,Virtual Families 3: Our Country Home,tbd
...,...,...
7695,Call of Duty: Ghosts,78
7696,FIFA 14,84
7697,Skylanders Swap Force,83
7698,NBA 2K14,84


# 3. ***Export*** do arquivo *csv*

In [19]:
metacritic['metascore'].unique()

array(['tbd', 84, 80, 85, 75, 90, 77, 86, 76, 88, 81, 78, 82, 83, 95, 100,
       92, 87, 96, 79, 91, 99, 89, 93, 97, 94, 98, 48], dtype=object)

In [21]:
metacritic.to_csv('xbox_one_metacritic.csv', index=False)